# Project D4: Predicting LoL winner

Team members - Veronika Kukk, Robin Piir, Otto Kase
<br>Github - https://github.com/VeronikaKukk/DS_project 

# Gathering data

Data was gathered from Riot API by using development API key.
<br> Code for that is written in request_info.py

In [1]:
# everything that needs to be imported
import pandas as pd
from pathlib import Path

# Combining data

In [2]:
# import all eune files to dataframes
df_eune1 = pd.read_csv("eune_data1.csv", index_col=0)
df_eune2 = pd.read_csv("eune_data2.csv", index_col=0)
df_eune3 = pd.read_csv("eune_data3.csv", index_col=0)
df_eune4 = pd.read_csv("eune_data4.csv", index_col=0)

In [3]:
# import all euw files to dataframes
df_euw1 = pd.read_csv("euw_data1.csv", index_col=0)
df_euw2 = pd.read_csv("euw_data2.csv", index_col=0)
df_euw3 = pd.read_csv("euw_data3.csv", index_col=0)
df_euw4 = pd.read_csv("euw_data4.csv", index_col=0)

In [4]:
df_eune1

,matchId,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueDragons,blueHeralds,blueEliteMonsters,...,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin,bluewins
0,EUN1_3250387583,118,4,1,4,3,5,0,0,0,...,14267,7.0,16956,193,72,8,-2314,21.436862,1584.661722,0
1,EUN1_3250345989,15,2,0,7,3,5,0,0,0,...,13815,6.4,15507,183,48,1397,893,20.325918,1534.440213,1
2,EUN1_3250132600,13,0,0,3,2,3,0,0,0,...,14202,7.0,17682,205,48,-544,-1343,22.768502,1577.357373,0
3,EUN1_3250109344,10,1,1,3,3,0,0,0,0,...,14684,6.6,16566,194,60,-541,111,21.549689,1631.111531,1
4,EUN1_3250074417,12,0,0,0,4,0,0,0,0,...,14784,6.6,16370,203,50,-2446,-2208,22.544534,1641.863978,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4063,EUN1_3241781223,112,1,1,12,4,10,1,0,1,...,14534,6.4,15108,175,52,3242,2265,19.439261,1614.458367,1
4064,EUN1_3241746052,17,5,1,13,6,20,0,0,0,...,15496,6.4,16230,195,64,3805,1745,21.659246,1721.188111,1
4065,EUN1_3241710887,16,1,0,3,9,3,0,0,0,...,17167,6.6,16519,171,56,-3654,-1748,18.995111,1906.953580,1
4066,EUN1_3241682864,34,1,0,4,7,2,0,0,0,...,14538,6.4,14915,171,36,568,1714,18.995181,1614.923621,0


In [5]:
# combining all of the dataframes into one
eune_dfs = [df_eune1, df_eune2, df_eune3, df_eune4]
euw_dfs = [df_euw1, df_euw2, df_euw3, df_euw4]

df_eune = pd.concat(eune_dfs, ignore_index=True)
df_euw = pd.concat(euw_dfs, ignore_index=True)

In [6]:
df_eune

,matchId,blueWardsPlaced,blueWardsDestroyed,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueDragons,blueHeralds,blueEliteMonsters,...,redTotalGold,redAvgLevel,redTotalExperience,redTotalMinionsKilled,redTotalJungleMinionsKilled,redGoldDiff,redExperienceDiff,redCSPerMin,redGoldPerMin,bluewins
0,EUN1_3250387583,118,4,1,4,3,5,0,0,0,...,14267,7.0,16956,193,72,8,-2314,21.436862,1584.661722,0
1,EUN1_3250345989,15,2,0,7,3,5,0,0,0,...,13815,6.4,15507,183,48,1397,893,20.325918,1534.440213,1
2,EUN1_3250132600,13,0,0,3,2,3,0,0,0,...,14202,7.0,17682,205,48,-544,-1343,22.768502,1577.357373,0
3,EUN1_3250109344,10,1,1,3,3,0,0,0,0,...,14684,6.6,16566,194,60,-541,111,21.549689,1631.111531,1
4,EUN1_3250074417,12,0,0,0,4,0,0,0,0,...,14784,6.6,16370,203,50,-2446,-2208,22.544534,1641.863978,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16246,EUN1_3245368080,75,4,1,6,6,8,0,0,0,...,15542,6.6,16303,194,56,-509,-1744,21.549649,1726.415723,1
16247,EUN1_3245264423,9,3,0,1,6,1,0,0,0,...,14910,6.4,16199,185,40,-1725,-1144,20.548097,1656.065576,0
16248,EUN1_3245007949,14,0,0,4,7,5,0,0,0,...,14705,6.6,16327,164,56,-501,-496,18.216353,1633.362583,1
16249,EUN1_3244978461,11,1,0,0,9,0,0,0,0,...,15390,6.2,15396,169,44,-2950,-967,18.771764,1709.452342,0


In [7]:
# write data to csv files
filepath1 = Path('combined_eune.csv')
filepath2 = Path('combined_euw.csv')
df_eune.to_csv(filepath1)
df_euw.to_csv(filepath2)

# Clean data